I Plan to use deep neural network to achieve better accuray. I will take 1 input layer 3 hidden layers and one final output layer.

nx -> 10 -> 10 -> 10 -> 1

In [1]:
import pickle
import matplotlib.pyplot as plt
import torch
import numpy as np
from torchmetrics import ConfusionMatrix
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:

def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data
dataDict = unpickle("../raw/cifar-10-batches-py/data_batch_1")

X=torch.tensor(dataDict[b'data'],device=device).T
Y=torch.tensor(dataDict[b'labels'],device=device).view(1,-1)
unique_values, counts = torch.unique(Y, return_counts=True)
# Print results
print("Unique values:", unique_values)
print("       Counts:", counts)

X=X.to(dtype=torch.float32)/255  # better to convert the range to 0 to 1 instead of 0 to 255 
m=len(Y[0])
nx = len(X)

m=len(Y[0])
nx = len(X)

# print(catsData)
print(f"labels:",list(set(dataDict[b'labels'])))
print(f" m: {m}")
print(f"nx: {nx}")
print(f"shape of X: {X.shape}")
print(f"shape of Y: {Y.shape}")

# fig,ax=plt.subplots()
# fig.set_size_inches(2,2)
# randomImageNum = torch.randint(0,m,(1,)).item()
# print(f"Random No: {randomImageNum}")
# ax.imshow((X[:,randomImageNum].view(-1,1024).T.view(32,32,3)+1)/2)
# ax.set_title(f"{Y[0,randomImageNum]}")
# ax.axis("off")

Unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], device='cuda:0')
       Counts: tensor([1005,  974, 1032, 1016,  999,  937, 1030, 1001, 1025,  981],
       device='cuda:0')
labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
 m: 10000
nx: 3072
shape of X: torch.Size([3072, 10000])
shape of Y: torch.Size([1, 10000])


In [3]:
def loss(y,A):
    y_pred_prob = A[y[0],torch.arange(A.size(1))]
    return -torch.log(y_pred_prob)
    # return (y*y_hat)**2
    return -(y*torch.log(A)+(1-y)*torch.log(1-A))

In [4]:
neurons = [nx,128,64,32,10]

print(f" m: {m}")
print(f"nx: {nx}")
print(f"shape of X: {X.shape}")
print(f"shape of Y: {Y.shape}")

## initialize weights and bias matrx
w0 = torch.rand(size=(neurons[0], neurons[1]),device=device) * 0.01
w1 = torch.rand(size=(neurons[1], neurons[2]),device=device) * 0.01
w2 = torch.rand(size=(neurons[2], neurons[3]),device=device) * 0.01
w3 = torch.rand(size=(neurons[3], neurons[4]),device=device) * 0.01
b0 = torch.rand(size=(neurons[1],1),device=device)*1.0
b1 = torch.rand(size=(neurons[2],1),device=device)*1.0
b2 = torch.rand(size=(neurons[3],1),device=device)*1.0
b3 = torch.rand(size=(neurons[4],1),device=device)*1.0

# Always put this at end 
w0.requires_grad_(True)
w1.requires_grad_(True)
w2.requires_grad_(True)
w3.requires_grad_(True)
b0.requires_grad_(True)
b1.requires_grad_(True)
b2.requires_grad_(True)
b3.requires_grad_(True)

z1 = w0.T @ X +b0
A1 = torch.relu(z1)

z2 = w1.T@A1 + b1
A2 = torch.relu(z2)

z3 = w2.T@A2 + b2
A3 = torch.relu(z3)

z4 = w3.T@A3 + b3
# A4 = torch.sigmoid(z4)
A4 = torch.exp(z4)/torch.sum(torch.exp(z4),dim=0)

L = loss(Y,A4)
J = torch.sum(L)/m
_,preds = torch.max(A4,dim=0)
unique_values, counts = torch.unique(preds, return_counts=True)
# Print results
print("Unique values:", unique_values)
print("       Counts:", counts)

print(f"value of J: {J}")

 m: 10000
nx: 3072
shape of X: torch.Size([3072, 10000])
shape of Y: torch.Size([1, 10000])
Unique values: tensor([2], device='cuda:0')
       Counts: tensor([10000], device='cuda:0')
value of J: 2.350301504135132


|                  | **Actual Positive** | **Actual Negative** |
|------------------|---------------------|---------------------|
| **Predicted Positive** | True Positive (TP)     | False Positive (FP)    |
| **Predicted Negative** | False Negative (FN)    | True Negative (TN)     |


In [7]:
# alpha=0.05 # for first 20000 iterations 
alpha=0.01 # For further iterations
confmat = ConfusionMatrix(task="binary",num_classes=2)
for i in range(20000,30000):
    J.backward()
    with torch.no_grad():  # Disable gradient tracking during update
        w0 -= alpha * w0.grad
        b0 -= alpha * b0.grad
        
        w1 -= alpha * w1.grad
        b1 -= alpha * b1.grad

        w2 -= alpha * w2.grad
        b2 -= alpha * b2.grad

        w3 -= alpha * w3.grad
        b3 -= alpha * b3.grad
        

        w0.grad.zero_()
        b0.grad.zero_()
        w1.grad.zero_()
        b1.grad.zero_()
        w2.grad.zero_()
        b2.grad.zero_()
        w3.grad.zero_()
        b3.grad.zero_()

    z1 = w0.T @ X +b0
    A1 = torch.relu(z1)

    z2 = w1.T@A1 + b1
    A2 = torch.relu(z2)

    z3 = w2.T@A2 + b2
    A3 = torch.relu(z3)

    z4 = w3.T@A3 + b3
    A4 = torch.exp(z4)/torch.sum(torch.exp(z4),dim=0)
    # A4 = torch.sigmoid(z4)
    
    L = loss(Y,A4)
    J = torch.sum(L)/m
    if i%200 == 0:

        Y_cpu=Y.cpu()
        A4_cpu=A4.cpu()
        _,preds = torch.max(A4_cpu,dim=0)
        unique_values, counts = torch.unique(preds, return_counts=True)
        # Print results
        print("\nUnique values:", unique_values)
        print("       Counts:", counts)
        accuracy=torch.sum(Y_cpu==preds.view(1,-1))/Y_cpu.numel()
        if accuracy>0.70:
            torch.save({
                'w0': w0, 
                'w1': w1,
                'w2': w2,
                'w3': w3,
                'b0': b0, 
                'b1': b1,
                'b2': b2,
                'b3': b3,
            }, f'./wbs/03-WsAndBs-iter_{i}-accuracy_{accuracy:.2f}.pt')
        print(f"Iteration: {i:<3d} Cost: {J:<6.2f} Accucary:{accuracy:.2f}")
        print(f"{preds[0:10]}")
        


Unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
       Counts: tensor([1188,  954,  770,  843,  833, 1132, 1065, 1192, 1002, 1021])
Iteration: 20000 Cost: 0.37   Accucary:0.88
tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])

Unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
       Counts: tensor([1027,  964, 1050,  991,  975,  921, 1041, 1015, 1037,  979])
Iteration: 20200 Cost: 0.25   Accucary:0.93
tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])

Unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
       Counts: tensor([1031,  964, 1047, 1001,  980,  917, 1036, 1012, 1032,  980])
Iteration: 20400 Cost: 0.23   Accucary:0.94
tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])

Unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
       Counts: tensor([1032,  967, 1049,  998,  983,  916, 1034, 1011, 1030,  980])
Iteration: 20600 Cost: 0.22   Accucary:0.94
tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])

Unique values: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
       Counts: tensor([1029,  967, 1046, 1005,  986,  914